# Google Notebook specific Stuff

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install pytorch-lightning==1.3.3
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl
!pip install scGeneFit
!pip install scanpy
!pip install anndata
# my version of smashpy that fixed imports, nothing else was changed
!pip install matplotlib>=3.3.2
!pip install shap>=0.37.0
!pip install xgboost>=1.3.3
!pip install imbalanced-learn>=0.7.0
!pip install numba>=0.51.2
!pip install harmonypy>=0.0.5
!git clone https://github.com/beelze-b/SmashFork.git
!git clone https://github.com/ahsv/RankCorr.git

     |████████████████████████████████| 808kB 5.4MB/s 
     |████████████████████████████████| 645kB 11.2MB/s 
     |████████████████████████████████| 10.6MB 18.2MB/s 
     |████████████████████████████████| 235kB 34.8MB/s 
     |████████████████████████████████| 829kB 36.0MB/s 
     |████████████████████████████████| 122kB 39.5MB/s 
     |████████████████████████████████| 1.3MB 34.0MB/s 
     |████████████████████████████████| 296kB 45.1MB/s 
     |████████████████████████████████| 143kB 53.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=4df15d409946823cafbac505be0f7314f2b9b93f80d8f0d412b3ffd27e6a9c7a
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uni

In [3]:
# Check GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [4]:
import sys
import os
# to import utils
sys.path.append(os.path.abspath('/content/gdrive/My Drive/Differentiable-Sparse-Subset-Selection/notebooks'))

# All the Imports

In [5]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder


from RankCorr.picturedRocks import Rocks
import SmashFork.smashpy as smashpy


import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part o

CPU times: user 22.6 s, sys: 1.22 s, total: 23.8 s
Wall time: 28.7 s


In [6]:
from os import path

In [7]:
model_save_path = '/content/gdrive/MyDrive/Differentiable-Sparse-Subset-Selection/data/{}/models/'
num_times = 5

  

## Accuracies and F1

In [8]:
data_locations = [
                  'CITE-seq',
                  'developmental_mouse',
                  'mouse_brain_broad',
                  'paul',
                  'zeisel'
]

In [9]:
result_axis = [
 ('L1 VAE', 'l1_vae_results_{}.npy', 'l1_vae_knn_results_{}.npy'),
 ('Smash RF', 'smash_rf_results_{}.npy', 'smash_rf_knn_results_{}.npy'),
 ('SMaSH DNN', 'smash_results_{}.npy', 'smash_knn_results_{}.npy'),
 ('RankCorr', 'rankcorr_results_{}.npy', 'rankcorr_knn_results_{}.npy'),
 ('Global Gate VAE', 'globalgate_vae_results_{}.npy', 'globalgate_vae_knn_results_{}.npy'),
 ('RunningGumbel VAE', 'runningstate_vae_results_{}.npy', 'runningstate_vae_knn_results_{}.npy'),
 ('RunningGumbel Classifier', 'runningstate_classifier_results_{}.npy', 'runningstate_classifier_knn_results_{}.npy'),
 ('Concrete VAE',  'concrete_vae_results_{}.npy', 'concrete_vae_knn_results_{}.npy')
]

## Random Forest

In [12]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,RunningGumbel VAE,RunningGumbel Classifier,Concrete VAE
CITE-seq,"(0.123, 0.845)","(0.069, 0.915)","(0.067, 0.917)","(0.069, 0.912)","(0.126, 0.837)","(0.093, 0.882)","(0.063, 0.924)","(0.124, 0.838)"
developmental_mouse,"(0.354, 0.622)","(0.138, 0.858)","(0.596, 0.392)","(0.135, 0.861)","(0.335, 0.661)","(0.281, 0.697)","(0.088, 0.908)","(0.438, 0.533)"
mouse_brain_broad,"(0.148, 0.841)","(0.018, 0.982)","(0.014, 0.985)","(0.03, 0.969)","(0.09, 0.904)","(0.011, 0.989)","(0.005, 0.995)","(0.163, 0.826)"
paul,"(0.534, 0.397)","(0.116, 0.881)","(0.192, 0.797)","(0.147, 0.847)","(0.362, 0.598)","(0.13, 0.868)","(0.139, 0.858)","(0.412, 0.541)"
zeisel,"(0.198, 0.785)","(0.052, 0.947)","(0.066, 0.933)","(0.053, 0.946)","(0.193, 0.795)","(0.097, 0.898)","(0.047, 0.952)","(0.187, 0.801)"


## KNN

In [13]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,RunningGumbel VAE,RunningGumbel Classifier,Concrete VAE
CITE-seq,"(0.178, 0.798)","(0.073, 0.915)","(0.067, 0.921)","(0.081, 0.905)","(0.2, 0.77)","(0.116, 0.874)","(0.062, 0.929)","(0.182, 0.789)"
developmental_mouse,"(0.519, 0.475)","(0.227, 0.775)","(0.696, 0.277)","(0.208, 0.782)","(0.477, 0.513)","(0.396, 0.589)","(0.115, 0.883)","(0.577, 0.411)"
mouse_brain_broad,"(0.232, 0.742)","(0.028, 0.972)","(0.014, 0.985)","(0.04, 0.96)","(0.138, 0.851)","(0.014, 0.986)","(0.006, 0.994)","(0.238, 0.74)"
paul,"(0.589, 0.37)","(0.212, 0.772)","(0.294, 0.677)","(0.249, 0.739)","(0.416, 0.553)","(0.244, 0.747)","(0.226, 0.766)","(0.477, 0.488)"
zeisel,"(0.253, 0.735)","(0.054, 0.945)","(0.067, 0.931)","(0.056, 0.943)","(0.278, 0.707)","(0.12, 0.876)","(0.037, 0.963)","(0.273, 0.712)"


In [16]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m1, f11, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m2, f12, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m1-m2, 3)
    f1 = round(f11-f12, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results3 = pd.DataFrame(results)
tabulated_results3

,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,RunningGumbel VAE,RunningGumbel Classifier,Concrete VAE
CITE-seq,"(-0.055, 0.047)","(-0.004, 0.001)","(0.0, -0.003)","(-0.012, 0.007)","(-0.074, 0.067)","(-0.023, 0.008)","(0.001, -0.005)","(-0.058, 0.049)"
developmental_mouse,"(-0.165, 0.147)","(-0.088, 0.083)","(-0.1, 0.115)","(-0.073, 0.079)","(-0.142, 0.148)","(-0.115, 0.108)","(-0.027, 0.025)","(-0.138, 0.121)"
mouse_brain_broad,"(-0.084, 0.1)","(-0.01, 0.01)","(0.0, -0.0)","(-0.01, 0.01)","(-0.049, 0.053)","(-0.003, 0.003)","(-0.0, 0.0)","(-0.075, 0.087)"
paul,"(-0.054, 0.027)","(-0.096, 0.109)","(-0.102, 0.12)","(-0.101, 0.108)","(-0.055, 0.045)","(-0.114, 0.121)","(-0.087, 0.093)","(-0.066, 0.053)"
zeisel,"(-0.055, 0.05)","(-0.002, 0.002)","(-0.002, 0.002)","(-0.002, 0.003)","(-0.085, 0.088)","(-0.024, 0.022)","(0.011, -0.011)","(-0.086, 0.089)"


# Things to note for paper

PySmash uses a seed internally so the model is deterministic to a particular dataset. We used various different train seeds, whereas the smash paper uses the whole dataset for the data.